# Calculate VDK Flux using a pre-defined L-shell

## Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numba import jit
import xarray as xr

## Define a few functions to help
The main two functions include `gen_energy_grid` which generates an energy grid for us to use, as well as `vdk2016` which is explained in `apeep.ipynb`

In [2]:
# Generate a grid of energies for the flux spectrum
# The energy range of the spectrum was set at 30–1000 keV, 
# with nbins of logarithmically spaced grid points.

@jit(nopython=True)
def gen_energy_grid(nbins):

    e1 = np.log10(30.)
    e2 = np.log10(1000.)
    e = 10**(e1 + (e2-e1)*np.arange(nbins)/(nbins-1))
    
    return e

In [3]:
@jit(nopython=True)
def vdk2016(e, l, Ap):

    lpp = -0.7430*np.log(Ap) + 6.5257
    Spp = l - lpp

    # vdK2016 eqn.(8)

    A = 8.2091*Ap**0.16255
    b = 1.3754*Ap**0.33042
    c = 0.13334*Ap**0.42616
    s = 2.2833*Ap**-0.22990
    d = 2.7563e-4*Ap**2.6116

    # integral flux >30 keV (F30) electrons / (cm2 sr s)
    F30 = np.exp(A) / (np.exp(-b*(Spp-s)) + np.exp(c*(Spp-s)) + d)

    # vdK2016 eqn.(9)

    E = 3.3777*Ap**-1.7038 + 0.15
    bk = 3.7632*Ap**-0.16034
    sk = 12.184*Ap**-0.30111

    k = -1.0 / (E*np.exp(-bk*Spp) + 0.30450*np.cosh(0.20098*(Spp-sk))) - 1
    
    # solve eqn 3 for C
    # C is an offset, and k is the spectral gradient
    x=k+1
    c = F30*x/(1e3**x-30.**x)
    
    # calcualte the spectral density of the flux S(E) = CE^k
    # in electrons / (cm2 sr s keV)
    flux_spectral_density = e**k*c
    
    return flux_spectral_density

## Remotely pull in the data, and read it in using `pandas`

In [4]:
import urllib 

file, message = urllib.request.urlretrieve('ftp://ftp.gfz-potsdam.de/pub/home/obs/Kp_ap_Ap_SN_F107/Kp_ap_Ap_SN_F107_since_1932.txt', '../data/Kp_ap_Ap_SN_F107_since_1932.txt')

In [47]:
df = pd.read_csv(file, skiprows=39, header=0, delim_whitespace=True)

# Create a datetime index using the year, month, day columns
cols=["#YYY","MM","DD"]
df['date'] = df[cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")
df.index = pd.to_datetime(df.date)

# Use the 3-hourly data to calculate an average daily value, round to three decimal places
df['ap'] = df[['ap1', 'ap2', 'ap3', 'ap4', 'ap5', 'ap6', 'ap7', 'ap8']].mean(axis=1).round(3)
ap_df = df[['ap']]

In [48]:
ap_vals = ap_df.ap.values
times = ap_df.index.values

### Generate the energy grid and set of lshell values

In [49]:
nbins = 128 # sets resolution of the grid
e = gen_energy_grid(nbins)

l_shell = np.arange(2, 10.5, 0.5)

In [50]:
@jit(nopython=True)
def lshell_to_glat(lshell):
    return np.arccos((2.02/lshell) - 1) * (90./np.pi)

In [51]:
@jit(nopython=True)
def calculate_flux(lshell, aps, e=e):
    flux = np.empty(shape=(len(lshell),len(aps), len(e)))
    for i in range(len(lshell)):
        for j in range(len(aps)):
            # calculate the top of the atmosphere energetic electron energy spectrum
            
            if aps[j] != 0:
                flux_sd = vdk2016(e, lshell[i], aps[j])
                
            else:
                flux_sd = np.nan * e
            
            # van de Kamp is per steradian (electrons / (cm2 sr s keV))
            # assume flux is isotropic inside a nominal bounce loss cone (BLC) angle
            # of 80˚. The area of the BLC in sr is 2pi(1-cosd(66.3))
            flux[i, j, :] = 2.*np.pi*(1-np.cos(np.radians(80))) * flux_sd
            
    return flux

In [52]:
vdk_flux = calculate_flux(l_shell, ap_vals)

## Create a dataset from the output

In [53]:
ds = xr.Dataset(
    data_vars=dict(
        vdk_energy_spectrum=(["lshell", "time", "e"], vdk_flux),
        lshell=l_shell,
        glat = lshell_to_glat(l_shell),
        time=times,
        e=e,
    ),
    attrs=dict(description="Flux calculation"),
)

In [54]:
ds

<xarray.Dataset>
Dimensions:              (e: 128, glat: 17, lshell: 17, time: 32718)
Coordinates:
  * lshell               (lshell) float64 2.0 2.5 3.0 3.5 ... 8.5 9.0 9.5 10.0
  * glat                 (glat) float64 44.71 50.53 54.53 ... 70.43 70.97 71.47
  * time                 (time) datetime64[ns] 1932-01-01 ... 2021-07-29
  * e                    (e) float64 30.0 30.84 31.7 32.59 ... 946.3 972.8 1e+03
Data variables:
    vdk_energy_spectrum  (lshell, time, e) float64 0.05742 0.05584 ... 0.008549
Attributes:
    description:  Flux calculation

In [12]:
ds.to_netcdf('../data/vdk_flux_calc_test.nc')